In [30]:
import csv
import json
import requests

import glob
import os
from os.path import join

In [31]:
endpoint = 'https://www.loc.gov/free-to-use'
parameters = {
    'fo' : 'json'
}

In [32]:
collection = 'dogs'

In [33]:
collection_list_response = requests.get(endpoint + '/' + collection, params=parameters)

In [34]:
collection_list_response.url

'https://www.loc.gov/free-to-use/dogs?fo=json'

In [35]:
collection_json = collection_list_response.json()

In [36]:
collection_json.keys()

dict_keys(['breadcrumbs', 'content', 'content_is_post', 'description', 'expert_resources', 'next', 'next_sibling', 'options', 'pages', 'portal', 'previous', 'previous_sibling', 'site_type', 'timestamp', 'title', 'type'])

In [37]:
for k in collection_json['content']['set']['items']:
    print(k)

{'image': '/static/portals/free-to-use/public-domain/dogs/1_2590.jpg', 'link': '/resource/gottlieb.04271.0/', 'title': 'Portrait of Billie Holiday and Mister, Downbeat(?), New York, N.Y., ca. June 1946'}
{'image': '/static/portals/free-to-use/public-domain/dogs/2_1152.jpg', 'link': '/resource/sousa.200031428.0/', 'title': 'Sousa with dogs'}
{'image': '/static/portals/free-to-use/public-domain/dogs/3_3b21894r.jpg', 'link': '/resource/cph.3b21894/', 'title': 'Dogs - motoring'}
{'image': '/static/portals/free-to-use/public-domain/dogs/4_2773.jpg', 'link': '/resource/gottlieb.04241.0/', 'title': 'Portrait of Billie Holiday and Mister, Downbeat, New York, N.Y., ca. Feb. 1947'}
{'image': '/static/portals/free-to-use/public-domain/dogs/5_01592v.jpg', 'link': '/resource/ppmsc.01592/', 'title': 'Dogs pulling pupmobile'}
{'image': '/static/portals/free-to-use/public-domain/dogs/6_05664v.jpg', 'link': '/resource/ppmsc.05664/', 'title': 'Milksellers, Brussels, Belgium'}
{'image': '/static/portals/

In [38]:
len(collection_json['content']['set']['items'])

28

In [39]:
collection_json['content']['set']['items'][0].keys()

dict_keys(['image', 'link', 'title'])

In [40]:
collection_set_list = os.path.join('data','collection_set_list_rlk_0.csv')
headers = ['image','link','title']

with open(collection_set_list, 'w', encoding='utf-8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=headers)
    writer.writeheader()
    for item in collection_json['content']['set']['items']:
        
        # clean up errant spaces in the title fields
        item['title'] = item['title'].rstrip()
        writer.writerow(item)
    print('wrote',collection_set_list)

wrote data/collection_set_list_rlk_0.csv


In [41]:
# update endpoint info
endpoint = 'https://www.loc.gov'
parameters = {
    'fo' : 'json'
}

In [42]:
#   NAME CHANGE FOR FOLDER

# run this cell to confirm that you have a location for the JSON files
item_metadata_directory = os.path.join('data','ftu_dogs_metadata_rlk')

if os.path.isdir(item_metadata_directory):
    print(item_metadata_directory,'exists')
else:
    os.mkdir(item_metadata_directory)
    print('created',item_metadata_directory)

data/ftu_dogs_metadata_rlk exists


In [43]:
item_count = 0
error_count = 0
file_count = 0

data_directory = 'data'
item_metadata_directory = 'ftu_dogs_metadata_rlk'
item_metadata_file_start = 'item_metadata'
json_suffix = '.json'

collection_set_list = os.path.join('data','collection_set_list_rlk.csv')

with open(collection_set_list, 'r', encoding='utf-8', newline='') as f:
    reader = csv.DictReader(f, fieldnames=headers)
    for item in reader:
        if item['link'] == 'link':
            continue
        # these resource links could redirect to item pages, but currently don't work
        if '?' in item['link']:
            resource_ID = item['link']
            short_ID = item['link'].split('/')[2]
            item_metadata = requests.get(endpoint + resource_ID + '&fo=json')
            print('requested',item_metadata.url,item_metadata.status_code)
            if item_metadata.status_code != 200:
                print('requested',item_metadata.url,item_metadata.status_code)
                error_count += 1
                continue
            try:
                item_metadata.json()
            except: #basically this catches all of the highsmith photos with hhh in the ID
                error_count += 1
                print('no json found')
                continue
            fout = os.path.join(data_directory, item_metadata_directory, str(item_metadata_file_start + '-' + short_ID + json_suffix))
            with open(fout, 'w', encoding='utf-8') as json_file:
                json_file.write(json.dumps(item_metadata.json()['item']))
                file_count += 1
                print('wrote', fout)
            item_count += 1
        else:
            resource_ID = item['link']
            short_ID = item['link'].split('/')[2]
            item_metadata = requests.get(endpoint + resource_ID, params=parameters)
            print('requested',item_metadata.url,item_metadata.status_code)
            if item_metadata.status_code != 200:
                print('requested',item_metadata.url,item_metadata.status_code)
                error_count += 1
                continue
            try:
                item_metadata.json()
            except:
                error_count += 1
                print('no json found')
                continue
            fout = os.path.join(data_directory, item_metadata_directory, str(item_metadata_file_start + '-' + short_ID + json_suffix))
            with open(fout, 'w', encoding='utf-8') as json_file:
                json_file.write(json.dumps(item_metadata.json()['item']))
                file_count += 1
                print('wrote', fout)
            item_count += 1

print('--- mini LOG ---')
print('items requested:',item_count)
print('errors:',error_count)
print('files written:',file_count)

requested https://www.loc.gov/resource/gottlieb.04271.0/?fo=json 200
wrote data/ftu_dogs_metadata_rlk/item_metadata-gottlieb.04271.0.json
requested https://www.loc.gov/resource/sousa.200031428.0/?fo=json 200
wrote data/ftu_dogs_metadata_rlk/item_metadata-sousa.200031428.0.json
requested https://www.loc.gov/resource/cph.3b21894/?fo=json 200
wrote data/ftu_dogs_metadata_rlk/item_metadata-cph.3b21894.json
requested https://www.loc.gov/resource/gottlieb.04241.0/?fo=json 200
wrote data/ftu_dogs_metadata_rlk/item_metadata-gottlieb.04241.0.json
requested https://www.loc.gov/resource/ppmsc.01592/?fo=json 200
wrote data/ftu_dogs_metadata_rlk/item_metadata-ppmsc.01592.json
requested https://www.loc.gov/resource/ppmsc.05664/?fo=json 200
wrote data/ftu_dogs_metadata_rlk/item_metadata-ppmsc.05664.json
requested https://www.loc.gov/resource/ppmsc.05665/?fo=json 200
wrote data/ftu_dogs_metadata_rlk/item_metadata-ppmsc.05665.json
requested https://www.loc.gov/resource/cai.2a12829/?fo=json 200
wrote da

TRANSFORMATION PART 1: Testing

In [44]:
current_loc = os.getcwd()

print(current_loc)

/Users/raynaketchum/Documents/umich/SI676/networked-services-labs


In [45]:
metadata_file_path = os.path.join('data','ftu_dogs_metadata_rlk')

print(metadata_file_path)

data/ftu_dogs_metadata_rlk


In [46]:
file_count = 0

for file in glob.glob('data/ftu_dogs_metadata_rlk/item_metadata-*.json'):
    file_count += 1
    print(file)

print('found', file_count)

data/ftu_dogs_metadata_rlk/item_metadata-cai.2a12829.json
data/ftu_dogs_metadata_rlk/item_metadata-ggbain.11087.json
data/ftu_dogs_metadata_rlk/item_metadata-cph.3b21894.json
data/ftu_dogs_metadata_rlk/item_metadata-ggbain.21426.json
data/ftu_dogs_metadata_rlk/item_metadata-gottlieb.01111.0.json
data/ftu_dogs_metadata_rlk/item_metadata-ppmsc.05667.json
data/ftu_dogs_metadata_rlk/item_metadata-ggbain.24645.json
data/ftu_dogs_metadata_rlk/item_metadata-ggbain.21168.json
data/ftu_dogs_metadata_rlk/item_metadata-ppmsc.01592.json
data/ftu_dogs_metadata_rlk/item_metadata-ggbain.27023.json
data/ftu_dogs_metadata_rlk/item_metadata-hec.05628.json
data/ftu_dogs_metadata_rlk/item_metadata-sousa.200031428.0.json
data/ftu_dogs_metadata_rlk/item_metadata-ggbain.18296.json
data/ftu_dogs_metadata_rlk/item_metadata-ggbain.20728.json
data/ftu_dogs_metadata_rlk/item_metadata-ggbain.26131.json
data/ftu_dogs_metadata_rlk/item_metadata-gottlieb.04241.0.json
data/ftu_dogs_metadata_rlk/item_metadata-ggbain.17

In [47]:
list_of_item_metadata_files = list()
for file in glob.glob('data/ftu_dogs_metadata_rlk/item_metadata-*.json'):
    list_of_item_metadata_files.append(file)

In [48]:
len(list_of_item_metadata_files)

28

In [49]:
list_of_item_metadata_files.sort()

for file in list_of_item_metadata_files:
    print(file)

data/ftu_dogs_metadata_rlk/item_metadata-cai.2a12829.json
data/ftu_dogs_metadata_rlk/item_metadata-cph.3b21894.json
data/ftu_dogs_metadata_rlk/item_metadata-ggbain.10973.json
data/ftu_dogs_metadata_rlk/item_metadata-ggbain.11087.json
data/ftu_dogs_metadata_rlk/item_metadata-ggbain.11088.json
data/ftu_dogs_metadata_rlk/item_metadata-ggbain.11145.json
data/ftu_dogs_metadata_rlk/item_metadata-ggbain.16691.json
data/ftu_dogs_metadata_rlk/item_metadata-ggbain.17751.json
data/ftu_dogs_metadata_rlk/item_metadata-ggbain.18296.json
data/ftu_dogs_metadata_rlk/item_metadata-ggbain.19461.json
data/ftu_dogs_metadata_rlk/item_metadata-ggbain.20728.json
data/ftu_dogs_metadata_rlk/item_metadata-ggbain.21168.json
data/ftu_dogs_metadata_rlk/item_metadata-ggbain.21426.json
data/ftu_dogs_metadata_rlk/item_metadata-ggbain.24645.json
data/ftu_dogs_metadata_rlk/item_metadata-ggbain.26131.json
data/ftu_dogs_metadata_rlk/item_metadata-ggbain.27023.json
data/ftu_dogs_metadata_rlk/item_metadata-gottlieb.01111.0.

In [50]:
# try first with one file, can you open the json, can you see what elements are in the json?
with open(list_of_item_metadata_files[0], 'r', encoding='utf-8') as item:
    # what are we looking at?
    print('file:',list_of_item_metadata_files[0],'\n')
    
    # load the item data
    item_data = json.load(item)
    
    for element in item_data.keys():
        print(element,':',item_data[element])

file: data/ftu_dogs_metadata_rlk/item_metadata-cai.2a12829.json 

_version_ : 1709975129956548608
access_advisory : Restricted access: Materials in this collection are often extremely fragile; most originals cannot be served.
access_restricted : False
aka : ['http://www.loc.gov/pictures/collection/cai/item/2010716169/', 'http://www.loc.gov/item/2010716169/', 'http://www.loc.gov/pictures/item/2010716169/', 'http://hdl.loc.gov/loc.pnp/cai.2a12829', 'http://www.loc.gov/resource/cai.2a12829/', 'http://lccn.loc.gov/2010716169']
call_number : CAI - Gibson, no. 26 (D size) [P&P]
campaigns : []
contributor_names : ['Gibson, Charles Dana, 1867-1944, artist']
contributors : [{'gibson, charles dana': 'https://www.loc.gov/search/?fa=contributor:gibson,+charles+dana&fo=json'}]
control_number : 
created : 2016-04-20 12:39:14
created_published : ['[1903?]']
created_published_date : [1903?]
creator : Gibson, Charles Dana, 1867-1944, artist.
date : 1903
dates : [{'1903': 'https://www.loc.gov/search/?da

In [51]:
item_data.keys()

dict_keys(['_version_', 'access_advisory', 'access_restricted', 'aka', 'call_number', 'campaigns', 'contributor_names', 'contributors', 'control_number', 'created', 'created_published', 'created_published_date', 'creator', 'date', 'dates', 'description', 'digital_id', 'digitized', 'display_offsite', 'extract_timestamp', 'extract_urls', 'format', 'format_headings', 'group', 'hassegments', 'id', 'image_url', 'index', 'item', 'language', 'languages', 'library_of_congress_control_number', 'link', 'marc', 'medium', 'medium_brief', 'mime_type', 'modified', 'notes', 'online_format', 'original_format', 'other_formats', 'other_title', 'partof', 'related', 'repository', 'reproductions', 'resource_links', 'resources', 'restriction', 'rights', 'rights_advisory', 'rights_information', 'score', 'shelf_id', 'site', 'sort_date', 'source_collection', 'source_created', 'source_modified', 'subject', 'subject_headings', 'subjects', 'thumb_gallery', 'timestamp', 'title', 'type', 'unrestricted', 'url'])

In [52]:
print('\ndate:',item_data['date'], type(item_data['date']))



date: 1903 <class 'str'>


In [53]:
print('\nformat:',item_data['format'][0], type(item_data['format']))


format: {'photo, print, drawing': 'https://www.loc.gov/search/?fa=original_format:photo,+print,+drawing&fo=json'} <class 'list'>


TEST: TRY IT WITH ONE EXAMPLE 

In [54]:
collection_info_csv = 'collection_items_data_rlk.csv'

headers = ['source_file', 'item_id', 'title', 'date', 'source_url', 'phys_format', 'subjects', 'rights'
]

with open(list_of_item_metadata_files[0], 'r', encoding='utf-8') as data:
    item_data = json.load(data)

    source_file = str(file)
    try:
        item_id = item_data['library_of_congress_control_number']
    except:
        item_id = item_data['url'].split('/')[-2]
    title = item_data['title']
    date = item_data['date']
    source_url = item_data['url']
    try:
        phys_format = item_data['format'][0]
    except:
        phys_format = 'Not found'
    try:
        subjects = item_data['subjects'][0]
    except:
        subjects = 'Not found'
    mime_type = item_data['mime_type']
    try:
        rights = item_data['rights_information']
    except:
        rights = 'Undetermined'
    
    
    row_dict = dict()
    # look for the item metadata, assign it to the dictionary; 
    # start with some basic elements likely (already enumerated in the headers list) :
    # source file
    row_dict['source_file'] = source_file
    # identifier
    row_dict['item_id'] = item_id
    # title
    row_dict['title'] = title
    # date
    row_dict['date'] = date
    # link
    row_dict['source_url'] = source_url
    # format
    row_dict['phys_format'] = phys_format
    # digital format
    row_dict['subjects'] = subjects
    #rights
    row_dict['rights'] = rights
    print('created row dictionary:',row_dict)

    # write to the csv
    with open(collection_info_csv, 'w', encoding='utf-8') as fout:
        writer = csv.DictWriter(fout, fieldnames=headers)
        writer.writeheader()
        writer.writerow(row_dict)
        print('wrote',collection_info_csv)

created row dictionary: {'source_file': 'data/ftu_dogs_metadata_rlk/item_metadata-sousa.200031428.0.json', 'item_id': '2010716169', 'title': 'Some women prefer dogs', 'date': '1903', 'source_url': 'https://www.loc.gov/item/2010716169/', 'phys_format': {'photo, print, drawing': 'https://www.loc.gov/search/?fa=original_format:photo,+print,+drawing&fo=json'}, 'subjects': {'children': 'https://www.loc.gov/search/?fa=subject:children&fo=json'}, 'rights': 'No known restrictions on publication.'}
wrote collection_items_data_rlk.csv


Developing the structure of the CSV file 

Find a direct url to a good image file for the item. 

HEADERS!

In [55]:
collection_info_csv = 'collection_items_data_rlk.csv'

# set up a list for the columns in your csv; in future, this should be more automated but this works for now as you set up the crosswalk
headers = ['source_file', 'item_id', 'title', 'date', 'source_url', 'phys_format', 'subjects', 'rights']

# try first with one file
with open(list_of_item_metadata_files[0], 'r', encoding='utf-8') as data:
    # load the item data
    item_data = json.load(data)
    
    print(item_data['image_url'][3])

https://tile.loc.gov/storage-services/service/pnp/cai/2a12000/2a12800/2a12829r.jpg#h=626&w=640


Transformation Part 2

In [56]:
items_data_file = os.path.join(data_directory, 'collection_items_data.csv')

if os.path.isfile(items_data_file):
    os.unlink(items_data_file)
    print('removed',items_data_file)

# clear row_dict
row_dict = ()

removed data/collection_items_data.csv


In [57]:
from datetime import date

date_string_for_today = date.today().strftime('%Y-%m-%d') # see https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior

print(date_string_for_today)

2022-12-01


In [58]:
collection_info_csv = os.path.join('data','collection_items_data.csv')
file_count = 0
items_written = 0
error_count = 0

headers = ['item_type', 'date_uploaded', 'source_file', 'item_id', 'title', 'date', 'source_url', 'phys_format', 'subjects', 'rights', "image_url" ]

for file in list_of_item_metadata_files:
    file_count += 1
    print('opening',file)
    with open(file, 'r', encoding='utf-8') as item:
        # load the item data
        try:
            item_data = json.load(item)
        except:
            print('error loading',file)
            error_count += 1
            continue

        # extract/name the data you want
        # item type
        item_type = 'Item'
        # date uplaoded
        date_uploaded = date_string_for_today
        # for checking purposes, add in the source of the info
        source_file = str(file)
        # make sure there's some unique and stable identifier
        try:
            item_id = item_data['library_of_congress_control_number']
        except:
            item_id = item_data['url'].split('/')[-2]
        title = item_data['title']
        try:
            date = item_data['date']
        except:
            date = 'n.d.'
        source_url = item_data['url']
        try:
            phys_format = item_data['format'][0]
        except:
            phys_format = 'Not found'
        try:
            subjects = item_data['subjects'][0]
        except:
            subjects = 'Not found'
        try:
            rights = item_data['rights_information']
        except:
            rights = 'Undetermined'
        try:
            image_url = item_data['image_url'][0]
        except:
            image_url = 'Not Found'

         # dictionary for the rows
        row_dict = dict()

        # look for the item metadata, assign it to the dictionary;
        # start with some basic elements likely (already enumerated in the headers list) :
        # item type
        row_dict['item_type'] = item_type
        # date uploaded
        row_dict['date_uploaded'] = date_uploaded
        # source filename
        row_dict['source_file'] = source_file
        # identifier
        row_dict['item_id'] = item_id
        # title
        row_dict['title'] = title
        # date
        row_dict['date'] = date
        # link
        row_dict['source_url'] = source_url
        # format
        row_dict['phys_format'] = phys_format
        # digital format
        row_dict['subjects'] = subjects
        #rights
        row_dict['rights'] = rights
        #image
        row_dict['image_url'] = image_url


        # write to the csv
        with open(collection_info_csv, 'a', encoding='utf-8') as fout:
            writer = csv.DictWriter(fout, fieldnames=headers)
            if items_written == 0:
                writer.writeheader()
            writer.writerow(row_dict)
            items_written += 1
            print('adding',item_id)

print('\n\n--- LOG ---')
print('wrote',collection_info_csv)
print('with',items_written,'items')
print(error_count,'errors (info not written)')


opening data/ftu_dogs_metadata_rlk/item_metadata-cai.2a12829.json
adding 2010716169
opening data/ftu_dogs_metadata_rlk/item_metadata-cph.3b21894.json
adding 2003677476
opening data/ftu_dogs_metadata_rlk/item_metadata-ggbain.10973.json
adding 2014690955
opening data/ftu_dogs_metadata_rlk/item_metadata-ggbain.11087.json
adding 2014691174
opening data/ftu_dogs_metadata_rlk/item_metadata-ggbain.11088.json
adding 2014691175
opening data/ftu_dogs_metadata_rlk/item_metadata-ggbain.11145.json
adding 2014691232
opening data/ftu_dogs_metadata_rlk/item_metadata-ggbain.16691.json
adding 2014696845
opening data/ftu_dogs_metadata_rlk/item_metadata-ggbain.17751.json
adding 2014698005
opening data/ftu_dogs_metadata_rlk/item_metadata-ggbain.18296.json
adding 2014698333
opening data/ftu_dogs_metadata_rlk/item_metadata-ggbain.19461.json
adding 2014699393
opening data/ftu_dogs_metadata_rlk/item_metadata-ggbain.20728.json
adding 2014700662
opening data/ftu_dogs_metadata_rlk/item_metadata-ggbain.21168.json
